In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from Data.Library import Library
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
torch.cuda.get_device_name()

/home/jacobson.n/.conda/envs/bookcorpus/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Tesla V100-PCIE-32GB'

In [4]:
class S4Model(nn.Module):
    def __init__(self, vocab_size, d_model, d_internal, num_layers, device = torch.device('cpu')):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.d_internal = d_internal
        self.num_layers = num_layers
        self.device = device
        self.log_softmax = nn.LogSoftmax(-2)

        # Define model shape
        self.embeddings = nn.Embedding(self.vocab_size, self.d_model).to(self.device)
        self.S4_layers = nn.ModuleList()
        self.num_params = self.vocab_size*self.d_model
        for layer in range(self.num_layers):
            self.S4_layers.append(S4Layer(latent_size = self.d_internal, in_channels = self.d_model, out_channels = self.d_model, device=self.device))
            self.num_params += self.S4_layers[-1].num_params
        self.fc1 = nn.Linear(self.d_model, self.vocab_size).to(self.device)
    def forward(self, sequence, conv=True):
        batch_size, seq_length = sequence.shape
        x = self.embeddings(sequence.to(self.device))
        # Current shape is [batch_size, sequence_length, channels]
        x = x.permute(0, 2, 1)#.to(self.device)
         # Current shape is [batch_size, channels, sequence_length]
        for layer in self.S4_layers:
            x = layer(x, conv=conv)
            x = F.relu(x)
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        x = x.permute(0, 2, 1)
        x = self.log_softmax(x)
        x = x.to(torch.device('cpu')) 
        return x
class S4Layer(nn.Module):
    def __init__(self, latent_size=8, in_channels = 1, out_channels=10, device=torch.device('cpu')):
        super().__init__()
        self.latent_size = latent_size
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.device = device

         # Use hippo matrices for A and B
        self.A = self.gen_A(self.latent_size)
        self.B = self.gen_B(self.latent_size)
        self.C = nn.Linear(self.in_channels*self.latent_size, self.out_channels).to(self.device)
        self.D = nn.Linear(self.in_channels, self.out_channels).to(self.device)
        self.A_stack = None
        self.step = None
        self.A_bar = None
        self.B_bar = None
        self.log_softmax = nn.LogSoftmax(-1)
        self.num_params = self.in_channels*(self.latent_size + 1)*self.out_channels + self.latent_size**2 + self.latent_size # Last two terms include hippo matrices

    def gen_A(self, N):
        A = torch.zeros(N, N)
        for n in range(N):
            for k in range(N):
                if n > k:
                    A[n, k] = -(2*n + 1)**.5 * (2*k + 1)**.5
                elif n == k:
                    A[n, k] = -(n+1)
        return A
        
    def gen_B(self, N):
        B = torch.zeros(N)
        for n in range(N):
            B[n] = (2*n+1)**.5
        return B.unsqueeze(1)
        
    def discretize(self, step):
        if self.step == step and self.B_bar is not None and self.A_bar is not None:
            return self.A_bar, self.B_bar
        else:
            A = self.A.to('cpu')
            B = self.B.to('cpu')
            N = A.shape[0]
            I = torch.eye(N)
            A_bar = torch.linalg.solve_triangular(I - (step / 2.0) * A, (I + (step / 2.0) * A), upper=False).to(self.device)
            B_bar = torch.linalg.solve_triangular((I - (step / 2.0) * A), B * step, upper=False).to(self.device)
            self.A_bar = A_bar
            self.B_bar = B_bar                                                                                                        
            return self.A_bar, self.B_bar
    
    def get_legendre_kernel(self, seq_length):
        if self.A_stack is None:
            A_stack = torch.eye(self.latent_size).unsqueeze(-1).repeat((1, 1, seq_length)).permute(2, 0, 1).to(self.device)
            A_bar, B_bar = self.discretize(step=1.0/seq_length)
            for idx in range(0, seq_length):
                A_stack[:idx] = A_stack[:idx] @ A_bar
            A_stack = A_stack.permute(1, 2, 0).cpu()
            self.A_stack = A_stack
            return A_stack
        else:
            if seq_length > self.A_stack.shape[2]:
                print('recalcing kernel')
                self.A_stack = None
                return self.get_legendre_kernel(seq_length)
            else:
                return self.A_stack
            
    def get_legendre_rec(self, sequence):
        batch_size, in_channels, seq_length = sequence.shape
        x = torch.zeros([batch_size,self.latent_size, in_channels,  seq_length + 1]).to(self.device)
        A_bar, B_bar = self.discretize(step=1.0/seq_length)
        B_bar = B_bar.permute(1, 0).repeat(batch_size, 1).unsqueeze(2) # Shape of [batch_size, latent_sizem, 1]
        A_bar = A_bar.unsqueeze(0).repeat(batch_size, 1, 1)
        for idx in range(seq_length):
            seq = sequence[:,:,idx].unsqueeze(1)
            B_term = torch.matmul(B_bar, seq)
            x_term = x[:,:,:,idx]
            A_term = torch.matmul(A_bar, x_term)
            x[:, :, :, idx+1] = A_term + B_term
        # x has shape [batch_size, self.latent_size, in_channels, seq_length + 1]
        output = x[:, :, :, 1:]
        # output has shape [batch_size, self.latent_size, in_channels, seq_length]
        B_scale = self.B.unsqueeze(0).unsqueeze(2).to(self.device)
        # output has shape [batch_size, in_channels, latent_size, sequence_length]
        output = (B_scale * output).permute(0, 2, 1, 3)
        return output
    
    def get_legendre_conv(self, sequence):
        # Sequence of shape [batch_size, in_channels, seq_length]
        batch_size, in_channels, seq_length = sequence.shape
        A_stack = self.get_legendre_kernel(seq_length)
        A_bar, B_bar = self.discretize(step = 1.0/seq_length)
        # Apply B
        u = sequence
        u = B_bar.unsqueeze(0).unsqueeze(2) * u.unsqueeze(1)
        # Convolution
        K = A_stack.unsqueeze(2).to(self.device)
        output = F.conv2d(u, K, padding=(0, seq_length-1))[:, :, :, :seq_length]
        # output has shape [batch_size, self.latent_size, in_channels, seq_length]
        B_scale = self.B.unsqueeze(0).unsqueeze(2).to(self.device)
        # output has shape [batch_size, in_channels, latent_size, sequence_length]
        output = (B_scale * output).permute(0, 2, 1, 3)
        return output
    
    def forward(self, u, conv=True):
        # u has shape [batch_size, in_channels, sequence_length]
        if len(u.shape) == 2: # If 1 dimensional
            u = u.unsqueeze(1)
        if conv:
            output = self.get_legendre_conv(u.to(self.device))
        else:
            output = self.get_legendre_rec(u.to(self.device))
        # output has shape [batch_size, in_channels, latent_size, sequence_length]
        # output permuted to [batch_size, seq_length, in_channels, latent_size]
        # Flatten feature space to [batch_size, seq_length, in_channels * latent_size]
        output = output.permute(0, 3, 1, 2).flatten(2)
        output =  self.C(output) + self.D(u.permute(0, 2, 1))
        # output should now be of shape [batch_size, seq_length, out channels]
        return output.permute(0, 2, 1)

In [10]:
# Hyperparams
epochs = 64
lr = .001
seq_length=256
batch_size=64
d_model=128
d_internal=192
num_layers=2
train_size = 2**20
test_size = 2**16
encoding=76
vocab_size = 76
torch.manual_seed(0)
conv = True

# Setup
device = torch.device('cuda')
library = Library(encoding=encoding, train_size=train_size, test_size=test_size, download_new=False)
model = S4Model(vocab_size, d_model, d_internal, num_layers, device)
print(f'S4 Model with {model.num_params} parameters')
loss_fn = nn.NLLLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)
x_batch = torch.zeros([batch_size, seq_length-1])
y_batch = torch.zeros([batch_size, seq_length-1])
losses = torch.zeros(epochs)
perplexities = torch.zeros(epochs)
print('Training')
import time
tic = time.time()
# Training
dataloader = library.get_train_dataloader(seq_length)
for epoch in range(epochs):
    for idx, data in enumerate(dataloader):
        mod_idx = idx % batch_size
        if data.shape[0] != seq_length:
            break # End of usable dataloader
        x_batch[mod_idx] = data[:-1]
        y_batch[mod_idx] = data[1:]
        if mod_idx == batch_size-1:
            # Update weights
            optim.zero_grad()
            y_pred = model(x_batch.long(), conv=conv)
            loss = loss_fn(y_pred, y_batch.long())
            losses[epoch] += loss

            print(f'{epoch}:{idx+1}:{losses[epoch]:.4f}', end='\r')
            loss.backward()
            optim.step()
            
    # Test
    perplexities[epoch] = library.calc_perplexity(model, batch_size=64, seq_length=256)
    print(f'{epoch}:Total Loss:{losses[epoch]:.2f}:Perplexity:{perplexities[epoch]:.2f}')
print(time.time()-tic)

S4 Model with 8562688 parameters
Training


KeyboardInterrupt: 

In [13]:
torch.save(perplexities, 'S411Kperplexities.pt')
torch.save(model.state_dict(), f'Models/S411K.pkl')

In [17]:
torch.load('S411Kperplexities.pt')[-1]

/tmp/ipykernel_122971/3668444734.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('S411Kperplexities.pt')[-1]


tensor(7.7838)